In [8]:
! pip install pyspark

In [9]:
! pip install pyArrow

In [3]:
import pyspark
import pandas as pd
import seaborn as sb
from matplotlib import pyplot as plt
import pyspark.pandas as ps
import numpy as np
import sqlite3

In [4]:
connection = sqlite3.connect("database.sqlite")

In [5]:
cursor = connection.cursor()

# Joined tables player, player_college,  college, player_award, and team_half

In [6]:
query = "SELECT * FROM player INNER JOIN player_college USING(player_id) INNER JOIN player_award USING(player_id) INNER JOIN team_half USING(league_id) INNER JOIN college USING(college_id)"
#query = "SELECT * FROM player INNER JOIN player_college ON player.player_id = player_college.player_id INNER JOIN player_award ON player_college.player_id = player_award.player_id INNER JOIN team_half ON player_award.league_id = team_half.league_id INNER JOIN player_award_vote ON player_award.player_id=player_award_vote.player_id INNER JOIN college ON player_college.college_id = college.college_id"
cursor.execute(query)
result = cursor.fetchall()

In [7]:
pl_df=pd.read_sql(query, connection)

In [8]:
pl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102200 entries, 0 to 102199
Data columns (total 44 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   player_id      102200 non-null  object
 1   birth_year     102200 non-null  int64 
 2   birth_month    102200 non-null  int64 
 3   birth_day      102200 non-null  int64 
 4   birth_country  102200 non-null  object
 5   birth_state    102200 non-null  object
 6   birth_city     102200 non-null  object
 7   death_year     102200 non-null  object
 8   death_month    102200 non-null  object
 9   death_day      102200 non-null  object
 10  death_country  102200 non-null  object
 11  death_state    102200 non-null  object
 12  death_city     102200 non-null  object
 13  name_first     102200 non-null  object
 14  name_last      102200 non-null  object
 15  name_given     102200 non-null  object
 16  weight         102200 non-null  int64 
 17  height         102200 non-null  int64 
 18  bats

In [9]:
pl_df.head()

,player_id,birth_year,birth_month,birth_day,birth_country,birth_state,birth_city,death_year,death_month,death_day,...,div_id,div_win,rank,g,w,l,name_full,city,state,country
0,abbotji01,1967,9,19,USA,MI,Flint,,,,...,E,N,2,54,31,23,University of Michigan,Ann Arbor,MI,USA
1,abbotji01,1967,9,19,USA,MI,Flint,,,,...,E,N,4,51,28,23,University of Michigan,Ann Arbor,MI,USA
2,abbotji01,1967,9,19,USA,MI,Flint,,,,...,E,N,5,56,30,26,University of Michigan,Ann Arbor,MI,USA
3,abbotji01,1967,9,19,USA,MI,Flint,,,,...,E,N,2,52,29,23,University of Michigan,Ann Arbor,MI,USA
4,abbotji01,1967,9,19,USA,MI,Flint,,,,...,W,N,4,60,31,29,University of Michigan,Ann Arbor,MI,USA


In [10]:
convert_dict = {'birth_year': 'string', 'birth_month': 'string', 'birth_day': 'string', 'death_year': 'string', 
                'death_month': 'string', 'death_day': 'string', 'birth_country': 'string', 'birth_state': 'string',  
                'birth_city': 'string', 'death_country': 'string', 'death_state': 'string', 'death_city': 'string',
                'name_first': 'string', 'name_last': 'string'}
 
pl_df = pl_df.astype(convert_dict)

pl_df['birth_date'] = pl_df["birth_year"]+'-'+pl_df['birth_month']+'-'+pl_df["birth_day"]
pl_df['death_date'] = pl_df["death_year"]+'-'+pl_df['death_month']+'-'+pl_df["death_day"]
pl_df['birth_date'] = pd.to_datetime(pl_df['birth_date'])
pl_df['death_date'] = pd.to_datetime(pl_df['death_date'], errors='coerce')
pl_df['birth_location'] = pl_df['birth_city']+', '+pl_df['birth_state']+' '+pl_df['birth_country']
pl_df['death_location'] = pl_df['death_city']+', '+pl_df['death_state']+' '+pl_df['death_country']
pl_df['birth_name'] = pl_df['name_first']+' '+pl_df['name_last']

In [11]:
pl_df = pl_df.rename(columns={"name_full": "college_name"})

In [12]:
pl_df = pl_df.drop(columns=['birth_year', 'birth_month', 'birth_day', 'death_year', 'death_month', 'death_day', 'birth_city', 'birth_state', 'birth_country', 'death_city',
                   'death_state', 'death_country', 'name_first', 'name_last', 'retro_id', 'bbref_id'])
pl_df['death_location'] = pl_df['death_location'].replace({',  ': ''})

In [13]:
pd.set_option('display.max_columns', None)
pl_df.head()

,player_id,name_given,weight,height,bats,throws,debut,final_game,college_id,year,award_id,year,league_id,tie,notes,year,team_id,half,div_id,div_win,rank,g,w,l,college_name,city,state,country,birth_date,death_date,birth_location,death_location,birth_name
0,abbotji01,James Anthony,200,75,L,L,1989-04-08,1999-07-21,michigan,1986,TSN All-Star,1991,AL,,LHP,1981,BAL,1,E,N,2,54,31,23,University of Michigan,Ann Arbor,MI,USA,1967-09-19,NaT,"Flint, MI USA",,Jim Abbott
1,abbotji01,James Anthony,200,75,L,L,1989-04-08,1999-07-21,michigan,1986,TSN All-Star,1991,AL,,LHP,1981,BAL,2,E,N,4,51,28,23,University of Michigan,Ann Arbor,MI,USA,1967-09-19,NaT,"Flint, MI USA",,Jim Abbott
2,abbotji01,James Anthony,200,75,L,L,1989-04-08,1999-07-21,michigan,1986,TSN All-Star,1991,AL,,LHP,1981,BOS,1,E,N,5,56,30,26,University of Michigan,Ann Arbor,MI,USA,1967-09-19,NaT,"Flint, MI USA",,Jim Abbott
3,abbotji01,James Anthony,200,75,L,L,1989-04-08,1999-07-21,michigan,1986,TSN All-Star,1991,AL,,LHP,1981,BOS,2,E,N,2,52,29,23,University of Michigan,Ann Arbor,MI,USA,1967-09-19,NaT,"Flint, MI USA",,Jim Abbott
4,abbotji01,James Anthony,200,75,L,L,1989-04-08,1999-07-21,michigan,1986,TSN All-Star,1991,AL,,LHP,1981,CAL,1,W,N,4,60,31,29,University of Michigan,Ann Arbor,MI,USA,1967-09-19,NaT,"Flint, MI USA",,Jim Abbott


In [14]:
pl_df.columns = ['player_id', 'name_given', 'weight', 'height', 'bats', 'throws', 'debut', 'final_game', 'college_id', 'college_year', 'award_id', 'award_year',
                     'league_id', 'tie', 'notes', 'league_year', 'team_id', 'half', 'div_id', 'div_win', 'rank', 'g', 'w', 'l', 'college_name', 'college_city', 
                 'college_state', 'college_country', 'birth_date', 'death_date', 'birth_location', 'death_location', 'birth_name']


In [15]:
pl_df['college_location'] = pl_df['college_city']+', '+pl_df['college_state']+' '+pl_df['college_country']

# Player Details

In [16]:
pl_detail = pl_df[["player_id","birth_name","birth_date","name_given","height","weight"]].drop_duplicates()
pl_detail

,player_id,birth_name,birth_date,name_given,height,weight
0,abbotji01,Jim Abbott,1967-09-19,James Anthony,75,200
84,ageeto01,Tommie Agee,1942-08-09,Tommie Lee,71,195
164,akerja01,Jack Aker,1940-07-13,Jackie Delane,74,190
192,allisbo01,Bob Allison,1934-07-11,William Robert,75,205
220,aloumo01,Moises Alou,1966-07-03,Moises Rojas,75,185
...,...,...,...,...,...,...
101308,younger01,Eric Young,1967-05-18,Eric Orlando,69,180
101452,youngmi02,Michael Young,1976-10-19,Michael Brian,73,200
101676,zahnge01,Geoff Zahn,1945-12-19,Geoffrey Clayton,73,180
101732,zimmery01,Ryan Zimmerman,1984-09-28,Ryan Wallace,75,220


# Player College Details

In [17]:
pcd=pl_df[["player_id","name_given","college_id","college_name","college_location"]].drop_duplicates()
pcd

,player_id,name_given,college_id,college_name,college_location
0,abbotji01,James Anthony,michigan,University of Michigan,"Ann Arbor, MI USA"
84,ageeto01,Tommie Lee,grambling,Grambling State University,"Grambling, LA USA"
164,akerja01,Jackie Delane,casequo,College of the Sequoias,"Visalia, CA USA"
192,allisbo01,William Robert,kansas,University of Kansas,"Lawrence, KS USA"
220,aloumo01,Moises Rojas,cacanad,Canada College,"Redwood City, CA USA"
...,...,...,...,...,...
101676,zahnge01,Geoffrey Clayton,michigan,University of Michigan,"Ann Arbor, MI USA"
101732,zimmery01,Ryan Wallace,virginia,University of Virginia,"Charlottesville, VA USA"
101948,zitoba01,Barry William,capierc,Los Angeles Pierce College,"Woodland Hills, CA USA"
102032,zitoba01,Barry William,ucsb,"University of California, Santa Barbara","Santa Barbara, CA USA"


# Players awards won during the years

In [18]:
papy = pl_df[["player_id","award_id","award_year"]].drop_duplicates()
papy

,player_id,award_id,award_year
0,abbotji01,TSN All-Star,1991
84,ageeto01,Gold Glove,1966
112,ageeto01,Gold Glove,1970
136,ageeto01,Rookie of the Year,1966
164,akerja01,TSN Fireman of the Year,1966
...,...,...,...
101756,zimmery01,Silver Slugger,2009
101780,zimmery01,Silver Slugger,2010
101948,zitoba01,Cy Young Award,2002
101976,zitoba01,TSN All-Star,2002


## query 1: Get the top 5 players with the most awards over the years

In [20]:
p_award_count = papy.groupby('player_id')['award_id'].count().reset_index(name='award_count').sort_values('award_count', ascending=False)
p_award_count = p_award_count.head(5)

In [58]:
p_award_count

,player_id,award_count
33,bondsba01,44
361,schmimi01,31
233,larkiba01,22
319,piazzmi01,21
374,smithoz01,20


## query 2: Top 5 colleges with the most players

In [21]:
count_players = pcd.groupby('college_name')['college_name'].count().reset_index(name='count_of_players').sort_values('count_of_players', ascending=False)
count_players = count_players.head(5)

In [22]:
count_players

,college_name,count_of_players
1,Arizona State University,14
228,University of Michigan,11
246,University of Southern California,9
205,"University of California, Los Angeles",8
239,University of Notre Dame,7


## query 3: Find the top 5 Awards given out the most for all years

In [29]:
awarded_count = papy.groupby('award_id')['award_id'].count().reset_index(name='awards_given').sort_values('awards_given', ascending=False)
awarded_count = awarded_count.head(5)

In [30]:
awarded_count

,award_id,awards_given
5,Gold Glove,356
14,TSN All-Star,356
13,Silver Slugger,236
2,Baseball Magazine All-Star,235
12,Rookie of the Year,60
